This file uses a Python2 library from Github (https://github.com/vadimkantorov/caffemodel2pytorch) to convert Caffe models into PyTorch. The original model was trained with Caffe. Remarkably, you don't ever have to install caffe for it to work!

Note this script is also in Python2.

Run `merge_npys` to combine the files created in this script.

In [1]:
from caffemodel2pytorch import *

In [2]:
model = Net(
    prototxt = 'predicting_poverty_deploy.prototxt',
    weights = 'predicting_poverty_trained.caffemodel',
    caffe_proto = 'https://raw.githubusercontent.com/BVLC/caffe/master/src/caffe/proto/caffe.proto'
)

caffemodel2pytorch: loading model from [predicting_poverty_trained.caffemodel] in HDF5 format failed [Unable to open file (file signature not found)], falling back to caffemodel format
caffemodel2pytorch: loaded model from [predicting_poverty_trained.caffemodel] in caffemodel format


In [3]:
model

Net(
  (conv1): Convolution(3, 64, kernel_size=(11, 11), stride=(4, 4))
  (relu1): ReLU()
  (norm1): LocalResponseNorm(5, alpha=0.000500000023749, beta=0.75, k=1.0)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Convolution(64, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (norm2): LocalResponseNorm(5, alpha=0.000500000023749, beta=0.75, k=1.0)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Convolution(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (conv4): Convolution(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (conv5): Convolution(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu5): ReLU()
  (pool5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv6): Convolution(256, 4096, kernel_size=(6, 6), stride=(6, 6))
  (relu6): ReLU()
  (conv7

In [4]:
# we strip the last layers, so the output is just what was at the conv7 layer
# the paper uses the data at this layer as the "features" for the image
del model.prob, model.pool6, model.conv8, model.relu7

In [5]:
model.eval()
torch.set_grad_enabled(False)

In [6]:
# use CPU for detection
device = torch.device("cpu")

In [7]:
from torchvision import datasets, models, transforms

In [8]:
# turn the data into a PyTorch Tensor
data_transforms = {
    'transform': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

In [9]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# takes a 
def filename_to_im_tensor(file):
    im = plt.imread(file)[:,:,:3]
    im = (im*256)
    im -= np.array([103.334, 107.8797, 107.4072])
    # convert to BGR, their Github code says to do this
    input_img = im[:, :, [2, 1, 0]]
    # convert to D,H,W
    input_img = np.transpose(input_img, [2, 0, 1])
    # subtract the mean to normalize the image, their Github code says to do this
    mean_bgr = [103.334, 107.8797, 107.4072]
    for i in range(0, 3):
        input_img[i, :, :] = input_img[i, :, :] - mean_bgr[i]
    
    im = Image.fromarray(im.astype(np.uint8))
    im = data_transforms['transform'](im)
    return im[None].to(device)

In [10]:
import os
ims = os.listdir('../famine_data/data/ims_malawi_2016/')

In [11]:
len(ims)

28113

In [12]:
# I split up computation because sometimes the script would fail
# doing it in pieces helps
a = np.zeros((5000,4096))
b = np.zeros((5000,4096))
c = np.zeros((5000,4096))
d = np.zeros((5000,4096))
e = np.zeros((5000,4096))
f = np.zeros((3113,4096))

In [ ]:
for i in range(0,5000):
    im = filename_to_im_tensor('../../Predicting-Famine/famine_data/ims_malawi_2016/{}'.format(ims[i]))
    a[i,:] = np.squeeze(model(im)['conv7'].numpy())
    if i % 500 == 0:
        print i
        
np.save('forward_feats_5000.npy', a)
        
for i in range(5000,10000):
    im = filename_to_im_tensor('../../Predicting-Famine/famine_data/ims_malawi_2016/{}'.format(ims[i]))
    b[i-5000,:] = np.squeeze(model(im)['conv7'].numpy())
    if i % 500 == 0:
        print i

np.save('forward_feats_5000_10000.npy', b)
        
for i in range(10000,15000):
    im = filename_to_im_tensor('../../Predicting-Famine/famine_data/ims_malawi_2016/{}'.format(ims[i]))
    c[i-10000,:] = np.squeeze(model(im)['conv7'].numpy())
    if i % 500 == 0:
        print i
        
np.save('forward_feats_10000_15000.npy', c)
        
for i in range(15000,20000):
    im = filename_to_im_tensor('../../Predicting-Famine/famine_data/ims_malawi_2016/{}'.format(ims[i]))
    d[i-15000,:] = np.squeeze(model(im)['conv7'].numpy())
    if i % 500 == 0:
        print i
        
np.save('forward_feats_15000_20000.npy', d)
        
for i in range(20000,25000):
    im = filename_to_im_tensor('../../Predicting-Famine/famine_data/ims_malawi_2016/{}'.format(ims[i]))
    e[i-20000,:] = np.squeeze(model(im)['conv7'].numpy())
    if i % 500 == 0:
        print i
        
np.save('forward_feats_20000_25000.npy', e)
        
for i in range(25000,len(ims)):
    im = filename_to_im_tensor('../../Predicting-Famine/famine_data/ims_malawi_2016/{}'.format(ims[i]))
    f[i-25000,:] = np.squeeze(model(im)['conv7'].numpy())
    if i % 500 == 0:
        print i
        
np.save('forward_feats_final.npy', f)

0
500
1000
1500
2000
2500
3000
